In [22]:
# importing libraries to use
import pandas as pd 
from datetime import datetime

In [23]:
df_bikes = pd.read_csv("./Trip_Data_022023.csv")

# Let's take a look at the data we're dealing with:
print("Cyclistic's dataframe has", df_bikes.shape[0], "rows and", df_bikes.shape[1], "columns")

Cyclistic's dataframe has 190445 rows and 13 columns


In [24]:
# Let's take a quick look at the first 5 rows of the dataframe:
df_bikes.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,CBCD0D7777F0E45F,classic_bike,2023-02-14 11:59:42,2023-02-14 12:13:38,Southport Ave & Clybourn Ave,TA1309000030,Clark St & Schiller St,TA1309000024,41.920771,-87.663712,41.907993,-87.631501,casual
1,F3EC5FCE5FF39DE9,electric_bike,2023-02-15 13:53:48,2023-02-15 13:59:08,Clarendon Ave & Gordon Ter,13379,Sheridan Rd & Lawrence Ave,TA1309000041,41.957879,-87.649584,41.969517,-87.654691,casual
2,E54C1F27FA9354FF,classic_bike,2023-02-19 11:10:57,2023-02-19 11:35:01,Southport Ave & Clybourn Ave,TA1309000030,Aberdeen St & Monroe St,13156,41.920771,-87.663712,41.880419,-87.655519,member
3,3D561E04F739CC45,electric_bike,2023-02-26 16:12:05,2023-02-26 16:39:55,Southport Ave & Clybourn Ave,TA1309000030,Franklin St & Adams St (Temp),TA1309000008,41.920873,-87.663733,41.879434,-87.635504,member
4,0CB4B4D53B2DBE05,electric_bike,2023-02-20 11:55:23,2023-02-20 12:05:48,Prairie Ave & Garfield Blvd,TA1307000160,Cottage Grove Ave & 63rd St,KA1503000054,41.794827,-87.618795,41.780531,-87.605970,member


In [25]:
# We can also check for the type of data that there is in each column:
df_bikes.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [26]:
# Columns 'started at' and 'ended at' are not an object type, so, we should change that:
df_bikes['started_at'] = pd.to_datetime(df_bikes['started_at'])
df_bikes['ended_at'] = pd.to_datetime(df_bikes['ended_at'])

In [28]:
# We can now check again if it's changed or not:
df_bikes.dtypes

ride_id                       object
rideable_type                 object
started_at            datetime64[ns]
ended_at              datetime64[ns]
start_station_name            object
start_station_id              object
end_station_name              object
end_station_id                object
start_lat                    float64
start_lng                    float64
end_lat                      float64
end_lng                      float64
member_casual                 object
dtype: object

The types are correct in relation to info displayed in the dataframe.

In [36]:
# We have some columns that have different types of data in it that may be worth to look at.
print("The column 'rideable_type' has", df_bikes['rideable_type'].nunique(), "different types of bikes registered, which are",)


The column 'rideable_type' has 3 different types of bikes registered, which are


In [37]:
df_bikes['rideable_type'].value_counts()

electric_bike    98362
classic_bike     89888
docked_bike       2195
Name: rideable_type, dtype: int64

In [ ]:
df_bikes['ended_at']